### **1. Library**

In [15]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from anytree import Node

from torchCustom import CustomDataset, CustomLoss, CustomModel, CustomPrediction
from Evaluation import *

In [16]:
torch.manual_seed(123)

### **2. Tree**

In [17]:
reuters = Node("Reuters")

ccat = Node("CCAT", parent = reuters)
c11 = Node("C11", parent = ccat)
c21 = Node("C21", parent = ccat)
c24 = Node("C24", parent = ccat)

gcat = Node("GCAT", parent = reuters)
gcrim = Node("GCRIM", parent = gcat)
gdip = Node("GDIP", parent = gcat)
gpol = Node("GPOL", parent = gcat)
gvio = Node("GVIO", parent = gcat)

mcat = Node("MCAT", parent = reuters)
m12 = Node("M12", parent = mcat)
m14 = Node("M14", parent = mcat)
m141 = Node("M141", parent = m14)
m142 = Node("M142", parent = m14)

In [18]:
tree = reuters

### **3. Data**

In [19]:
dataset = CustomDataset(data_dir='../data/')

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [20]:
BATCH_SIZE = 2**5 # train_size

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False)

### **4. Train**

In [21]:
EPOCHS = 100
LEARNING_RATE = 0.001

In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
model = CustomModel(feature_length = dataset.X.shape[1],
                    output_size = len(tree.leaves)-1)
print(model)

CustomModel(
  (hidden): Linear(in_features=7205, out_features=64, bias=False)
  (tanh): Tanh()
  (output): Linear(in_features=64, out_features=9, bias=False)
)


In [24]:
criterion = CustomLoss(tree)
predictor = CustomPrediction(tree)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [25]:
for epoch in range(EPOCHS):
    print('\n'+'*'*50)
    print(f'Train Epoch {epoch+1}')
    
    for i, train_data in enumerate(train_loader, 0):
        train_X, train_y = train_data['X'], train_data['y']
 
        y_pred = model(train_X)
 
        loss = criterion(y_pred, train_y)
        print(f'Epoch { epoch + 1} | Batch : {i + 1} | Loss : {loss.item():.4f}')
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


**************************************************
Train Epoch 1
Epoch 1 | Batch : 1 | Loss : 4.2811
Epoch 1 | Batch : 2 | Loss : 4.0828
Epoch 1 | Batch : 3 | Loss : 4.0924
Epoch 1 | Batch : 4 | Loss : 4.1255
Epoch 1 | Batch : 5 | Loss : 4.3918
Epoch 1 | Batch : 6 | Loss : 4.0853
Epoch 1 | Batch : 7 | Loss : 4.1702
Epoch 1 | Batch : 8 | Loss : 4.1394
Epoch 1 | Batch : 9 | Loss : 4.0355
Epoch 1 | Batch : 10 | Loss : 3.9595
Epoch 1 | Batch : 11 | Loss : 4.0110
Epoch 1 | Batch : 12 | Loss : 4.0155

**************************************************
Train Epoch 2
Epoch 2 | Batch : 1 | Loss : 3.9297
Epoch 2 | Batch : 2 | Loss : 3.6569
Epoch 2 | Batch : 3 | Loss : 3.7069
Epoch 2 | Batch : 4 | Loss : 3.7484
Epoch 2 | Batch : 5 | Loss : 3.5793
Epoch 2 | Batch : 6 | Loss : 3.7434
Epoch 2 | Batch : 7 | Loss : 3.7196
Epoch 2 | Batch : 8 | Loss : 3.5498
Epoch 2 | Batch : 9 | Loss : 3.5563
Epoch 2 | Batch : 10 | Loss : 3.4183
Epoch 2 | Batch : 11 | Loss : 3.4994
Epoch 2 | Batch : 12 | Loss : 3.443

### **5. Evaluation**

In [26]:
results_ls = []

model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        X, y = data['X'], data['y']
        y_pred = predictor(model, X)
        # print(torch.cat((y, y_pred), dim=1))
        results_ls.append(CustomEvaluation(tree, y, y_pred))

In [27]:
import pandas as pd

results = pd.DataFrame(results_ls, columns=['0-1 loss', 'symloss', 'hloss1', 'hloss2', 'hp', 'hr', 'hf'])
results['accuracy'] = 1 - results['0-1 loss']
results

,0-1 loss,symloss,hloss1,hloss2,hp,hr,hf,accuracy
0,0.098901,0.241758,0.014042,0.012559,0.947867,0.947867,0.947867,0.901099
